# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [14]:
# Libraries
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [15]:
# your code here

admissions = pd.read_csv('https://raw.githubusercontent.com/ta-data-lis/lab-advanced-pandas/master/your-code/data/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [16]:
# your code here
a = list(admissions.columns)
a

b = [element.strip() for element in a]
b
admissions.columns = b
admissions.columns

del a,b

Let's evaluate the dataset by looking at the `head` function.

In [17]:
# your code here
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [18]:
# your code here
admissions.isnull().value_counts()

Serial No.  GRE Score  TOEFL Score  University Rating  SOP    LOR    CGPA   Research  Chance of Admit
False       False      False        False              False  False  False  False     False              385
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [19]:
# your code here
admissions.set_index('Serial No.', inplace=True)
admissions.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [20]:
# your code here
print(len(admissions))

print(len(admissions.groupby(['GRE Score', 'CGPA'])))


385
385


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [21]:
# your code here

admissions.set_index(['GRE Score', 'CGPA'], inplace=True)
admissions

,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92
316,8.00,104,3,3.0,3.5,1,0.72
322,8.67,110,3,3.5,2.5,1,0.80
314,8.21,103,2,2.0,3.0,0,0.65
330,9.34,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...
324,9.04,110,3,3.5,3.5,1,0.82
325,9.11,107,3,3.0,3.5,1,0.84
330,9.45,116,4,5.0,4.5,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [22]:
admissions.reset_index(inplace=True)
admissions

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...
380,324,9.04,110,3,3.5,3.5,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84
382,330,9.45,116,4,5.0,4.5,1,0.91
383,312,8.78,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [34]:
# your code here

admissions[admissions['CGPA'] > 9]

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
4,330,9.34,115,5,4.5,3.0,1,0.90
10,328,9.10,112,4,4.0,4.5,1,0.78
19,328,9.50,116,5,5.0,5.0,1,0.94
20,334,9.70,119,5,5.0,4.5,1,0.95
...,...,...,...,...,...,...,...,...
379,329,9.23,111,4,4.5,4.0,1,0.89
380,324,9.04,110,3,3.5,3.5,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84
382,330,9.45,116,4,5.0,4.5,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [35]:
# your code here

admissions[(admissions['CGPA'] > 9) & (admissions['SOP'] < 3.5)]

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,118,4,3.0,4.5,1,0.91
62,327,9.02,114,3,3.0,3.0,0,0.61
140,326,9.11,114,3,3.0,3.0,1,0.83
217,324,9.01,111,4,3.0,3.0,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [45]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here

    return col - col.mean()

# print(standardize(admissions['CGPA']))

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [49]:
# your code here

admissions['CGPA_std'] = standardize(admissions['CGPA'])
admissions['GRE_std'] = standardize(admissions['GRE Score'])
admissions['LOR_std'] = standardize(admissions['LOR'])

# it's also possible to use DataFrame.insert()
# or DataFrame.assign()

We will generate the decision choice at random using the code below. Please run the cell.

In [63]:
# Libraries
from random import choices

In [64]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [66]:
# your code here
# the function lookup is deprecated !!

# admissions.assign(deciding_column  = decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [62]:
# your code here

# admissions['deciding_column']

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [31]:
# your code here

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [77]:
# your code here
new_list = []
for col in admissions.columns:
  col = col.replace(' ','_')
  col = col.lower()
  new_list.append(col)

admissions.columns = new_list
admissions.columns

Index(['gre_score', 'cgpa', 'toefl_score', 'university_rating', 'sop', 'lor',
       'research', 'chance_of_admit', 'cgpa_std', 'gre_std', 'lor_std'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [79]:
# your code here

# adjusted_gre

# admissions[admissions['university_rating'] >= 4]